JDC: This is odd: If I integrate the protocol work outside of the CustomIntegrator, I don't see the energy grow when I am just changing lambda_alchemical_bonds_{old|new} at the same time

Script: /home/chodera/equilibrated_restraint_experiments/281/test_langevin.py

In [1]:
import openmm
from openmm import unit

prefix = '/home/chodera/equilibrated_restraint_experiments/281/'

def read(filename):
    print(f'Reading {filename}...')
    with open(filename, 'r') as infile:
        return openmm.XmlSerializer.deserialize(infile.read())
    
def test(lambda_param=None):
    
    system = read(f'{prefix}281_apo_system.xml')
    state = read(f'{prefix}281_apo_state.xml')

    forces = { force.getName() : force for (index, force) in enumerate(system.getForces()) }
    cvforce = forces['CustomCVForce']

    platform = openmm.Platform.getPlatformByName('CUDA')
    platform.setPropertyDefaultValue('Precision', 'mixed')
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
    
    print("************K_RMSD != 0************")
    print('Creating context...')
    
    temperature = 300 * unit.kelvin
    collision_rate = 1.0 / unit.picoseconds
    timestep = 4.0 * unit.femtoseconds
    kT = unit.AVOGADRO_CONSTANT_NA * unit.BOLTZMANN_CONSTANT_kB * temperature
    integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
    context = openmm.Context(system, integrator, platform)
    context.setState(state)

    print('Integrating...')
#     nsteps_total = 250000
#     nsteps = 5000
    nsteps = 25000
    lambda_ = 0.0
    keywords = ['bonds', 'angles', 'torsions', 'sterics', 'sterics_exceptions', 'electrostatics',
                 'electrostatics_exceptions']
    for keyword in keywords:
        context.setParameter(f'lambda_alchemical_{keyword}_old', 1.0 - lambda_)
        context.setParameter(f'lambda_alchemical_{keyword}_new', lambda_)
    protocol_work = 0.0
    error = 0.0
    nsteps_per_report = 1000
    for step in range(nsteps):
#         lambda_ = step / nsteps_total
        lambda_ = step / nsteps

        Uold = context.getState(getEnergy=True).getPotentialEnergy() / kT

        for keyword in keywords:
            context.setParameter(f'lambda_alchemical_{keyword}_old', 1.0 - lambda_)
            context.setParameter(f'lambda_alchemical_{keyword}_new', lambda_)
#         context.setParameter('lambda_alchemical_bonds_old', 1.0 - lambda_)
#         context.setParameter('lambda_alchemical_bonds_new', lambda_)

        Unew = context.getState(getEnergy=True).getPotentialEnergy() / kT

        protocol_work += (Unew - Uold)
        time = context.getTime()
        cvs = cvforce.getCollectiveVariableValues(context)
        rmsd = cvs[0]

        if step % nsteps_per_report == 0:
            print(f'step {step:10.0f} : time {time/unit.nanoseconds:10.3f} ns | lambda {lambda_:16.12f} | protocol_work = {protocol_work:12.3f} | RMSD = {rmsd:12.5f} nm')

        integrator.step(1)

    del context, integrator
    
    print("************K_RMSD == 0************")
    print('Creating context...')

    integrator = openmm.LangevinIntegrator(temperature, collision_rate, timestep)
    
    context = openmm.Context(system, integrator, platform)
    context.setState(state)
    
    print(f'Setting K_RMSD to 0')
    context.setParameter('K_RMSD', 0.0)

    print('Integrating...')
#     nsteps_total = 250000
#     nsteps = 5000
    nsteps = 25000
    lambda_ = 0.0
    keywords = ['bonds', 'angles', 'torsions', 'sterics', 'sterics_exceptions', 'electrostatics',
                 'electrostatics_exceptions']
    for keyword in keywords:
        context.setParameter(f'lambda_alchemical_{keyword}_old', 1.0 - lambda_)
        context.setParameter(f'lambda_alchemical_{keyword}_new', lambda_)
    protocol_work = 0.0
    error = 0.0
    nsteps_per_report = 1000
    
    for step in range(nsteps):
#         lambda_ = step / nsteps_total
        lambda_ = step / nsteps
        
        Uold = context.getState(getEnergy=True).getPotentialEnergy() / kT

        for keyword in keywords:
            context.setParameter(f'lambda_alchemical_{keyword}_old', 1.0 - lambda_)
            context.setParameter(f'lambda_alchemical_{keyword}_new', lambda_)

        Unew = context.getState(getEnergy=True).getPotentialEnergy() / kT

        protocol_work += (Unew - Uold)
        time = context.getTime()
        cvs = cvforce.getCollectiveVariableValues(context)
        rmsd = cvs[0]

        if step % nsteps_per_report == 0:
            print(f'step {step:10.0f} : time {time/unit.nanoseconds:10.3f} ns | lambda {lambda_:16.12f} | protocol_work = {protocol_work:12.3f} | RMSD = {rmsd:12.5f} nm')

        integrator.step(1)

    del context, integrator



## Run neq changing all lambas

In [2]:
test()

Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_system.xml...
Reading /home/chodera/equilibrated_restraint_experiments/281/281_apo_state.xml...
************K_RMSD != 0************
Creating context...
Integrating...
step          0 : time      0.000 ns | lambda   0.000000000000 | protocol_work =       -0.000 | RMSD =      0.00026 nm
step       1000 : time      0.004 ns | lambda   0.040000000000 | protocol_work =       -2.654 | RMSD =      0.06041 nm
step       2000 : time      0.008 ns | lambda   0.080000000000 | protocol_work =       -3.130 | RMSD =      0.07256 nm
step       3000 : time      0.012 ns | lambda   0.120000000000 | protocol_work =       -3.132 | RMSD =      0.08532 nm
step       4000 : time      0.016 ns | lambda   0.160000000000 | protocol_work =       -2.692 | RMSD =      0.10073 nm
step       5000 : time      0.020 ns | lambda   0.200000000000 | protocol_work =       -2.195 | RMSD =      0.10423 nm
step       6000 : time      0.024 ns | lambda   0.